# Batch Slip and Fall

In [3]:
__version__ = 1.5
__author__ = 'Ajay Bhargava'

In [4]:
from utils import VideoReaders, DetectorLoader
import numpy as np, pandas as pd, glob
import cv2
from skimage import measure, color
from SORT.sort import *
import warnings
from itertools import groupby

In [5]:
class ChainedAssignment:
    def __init__(self, chained=None):
        acceptable = [None, 'warn', 'raise']
        assert chained in acceptable, "chained must be in " + str(acceptable)
        self.swcw = chained

    def __enter__(self):
        self.saved_swcw = pd.options.mode.chained_assignment
        pd.options.mode.chained_assignment = self.swcw
        return self

    def __exit__(self, *args):
        pd.options.mode.chained_assignment = self.saved_swcw

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = DetectorLoader.YOLACT('./weights/yolact_resnet50_54_800000.pth', threshold = 0.11)

In [17]:
def detect_fall(path):
    '''
    Docstring for detect_slip_fall()

    Arguments
    -----------
    str(path): Takes a path and sends it to the function for inference and subsequent action

    Returns
    -----------
    output(dict): A dictionary with the output of the model {"Frame": frame, "BBOX": bbox} 
    which is the first frame and person position where the person fell down. 
    '''

    frame_provider = VideoReaders.VideoReader(path)
    length, shape = frame_provider.properties()

    output_inference = []
    frames = []

    for frame in frame_provider:
        frames.append(frame)
        c, s, bb, ma = model.predict(frame)
        idx = np.where(c == 0)
        pixelwise_arrays = []
        for item in idx:
            for n, qq in enumerate(item):
                pixelwise = np.zeros_like(ma[qq,...])
                pixelwise = ma[qq,...].astype(np.uint8)
                pixelwise[np.where(pixelwise == 1)] = n + 1
                pixelwise_arrays.append(pixelwise)

        filtered = []
        for array in pixelwise_arrays:
            if np.max(array) != 0:
                filtered.append(array)

        stacked = np.sum(filtered, axis = 0)
        output_inference.append(stacked)

    output = []
    for arry in output_inference:
        if len(arry.shape) < 2:
            output.append(np.zeros(shape).astype(np.uint8))
        else:
            output.append(arry)
    
    if len(output) != len(frames):
        raise ValueError("Frame Lengths are off.")

    minimal_dictionary = []
    for i in range(0, length):
        for region in measure.regionprops(output[i], color.rgb2gray(frames[i])):
            minimal_dictionary.append({
                "Frame": i, 
                "ID": region.label, 
                "BBOX": region.bbox
            })

    tracked_bboxes = []
    Sorter = Sort(max_age = length, min_hits = 1, iou_threshold = 0.01) # 1 Tunable parameter (iou_threshold)
    for instance, entry in groupby(minimal_dictionary, key = lambda x:x['Frame']):
        entry_list = []
        for item in entry:
            lst = list(item['BBOX'])
            lst.extend([0, item['ID']])
            entry_list.append(lst)
        track_bbs_ids = Sorter.update(np.array(entry_list))
        for objects in track_bbs_ids:
            r0, c0, r1, c1, ID, label = objects.tolist()
            for region in measure.regionprops(output[instance], color.rgb2gray(frames[instance])):
                if region.label == label:
                    this_normalized_moment = region.moments_normalized
                    angle = np.degrees((np.arctan2(2*this_normalized_moment[1,1], this_normalized_moment[2,0] - this_normalized_moment[0,2]))/2)
                    w = region.bbox[2] - region.bbox[0]
                    h = region.bbox[3] - region.bbox[1]
                    ar = w / float(h)
                    tracked_bboxes.append({'Frame': instance, 
                                            'ID': int(label), 
                                            'Track ID': int(ID), 
                                            'BBOX': region.bbox,
                                            'Angle': angle,
                                            'Area': region.area,
                                            'Aspect Ratio': ar, 
                                            'Eccentricity': region.eccentricity, 
                                            'Perimeter': region.perimeter})
    tracked_dataframe = pd.DataFrame(tracked_bboxes)
    super_output = []
    if len(tracked_dataframe) == 0:
        super_output.append({'File': None, 'Frame': None, 'Bounding Box': None})
    else:
        for tracks, track_df in tracked_dataframe.groupby('Track ID'):
            current_track = track_df.loc[(track_df['Track ID'] == tracks), :]
            with ChainedAssignment():
                current_track['Rate of Change'] = current_track.loc[(current_track['Track ID'] == tracks), :]['Angle'].pct_change(5, fill_method = 'ffill') # 1 Tunable parameters (Periodicity for rate of change in angle)
                current_track['Slip-Fall'] = np.where(current_track['Rate of Change'] < -10, 'Slip', 'Stand') # 1 Tunable Parameter (Hard Cutoff for % change in angle)
            if len(current_track.loc[current_track['Slip-Fall'] == 'Slip']) != 0:
                super_output.append({'Track ID': tracks, 'File': os.path.basename(path), 'Frame': current_track.loc[current_track['Slip-Fall'] == 'Slip'].iloc[0]['Frame'], 'Bounding Box': current_track.loc[current_track['Slip-Fall'] == 'Slip'].iloc[0]['BBOX']})
            else:
                super_output.append({'Track ID': None, 'File': None, 'Frame': None, 'Bounding Box': None})
    df = pd.DataFrame(super_output)
    df = df.dropna()
    return df, tracked_dataframe

In [36]:
# Debugging
pathlist = sorted(glob.glob('../datasets/slip-fall/fall-database/*.mp4'))
for path in pathlist[33:34]:
    print(path)
    fall_df, track_df = detect_fall(path)

../datasets/slip-fall/fall-database/0034.mp4


In [37]:
fall_df

,Track ID,File,Frame,Bounding Box
0,151.0,0034.mp4,14.0,"(63, 327, 91, 349)"
1,152.0,0034.mp4,15.0,"(54, 220, 141, 333)"
2,153.0,0034.mp4,12.0,"(46, 280, 143, 377)"
3,154.0,0034.mp4,13.0,"(64, 313, 99, 347)"
4,155.0,0034.mp4,25.0,"(201, 267, 303, 423)"
6,157.0,0034.mp4,33.0,"(52, 257, 124, 417)"
11,164.0,0034.mp4,24.0,"(66, 393, 134, 417)"
12,165.0,0034.mp4,36.0,"(57, 321, 129, 402)"
23,179.0,0034.mp4,34.0,"(58, 345, 137, 365)"
39,197.0,0034.mp4,44.0,"(50, 215, 193, 283)"


In [54]:
track_df[(track_df['Track ID'] == 153) & (track_df['Frame'] >= 14)][['Frame', 'BBOX']].values.tolist()[0][1]

(57, 308, 131, 370)

In [ ]:
# fourcc = cv2.VideoWriter_fourcc(*'MP4V')
# writer = cv2.VideoWriter("./results/no-slip-videos/{}.mp4".format(video), 0x7634706d, 5.0, max([x.shape for x in boundaries])[:-1][::-1])
# for capture in boundaries:
#     out_capture = (capture * 255).astype(np.uint8)
#     writer.write(out_capture)
# writer.release()

# Add pixelwise outline + bounding box of fall detection when it happens. 

In [ ]:
# Fall Database
pathlist = glob.glob('../datasets/slip-fall/fall-database/*.mp4')
falls = []
for path in pathlist:
    falls.append(detect_fall(path))
flatten_fall = [x for x in falls for x in x]
df = pd.DataFrame(flatten_fall)
final = df.dropna()
fall_table = final[['File', 'Frame']].groupby(['File']).agg(Falls = ('Frame','count')).reset_index()

In [ ]:
# No Fall Database
pathlist = glob.glob('../datasets/slip-fall/no-fall-database/*.mp4')
falls = []
for path in pathlist:
    falls.append(detect_fall(path))
flatten_fall = [x for x in falls for x in x]
df = pd.DataFrame(flatten_fall)
final = df.dropna()
final[['File', 'Frame']].groupby(['File']).agg(Falls = ('Frame','count')).reset_index()